# Benchmarking mit LLM 
In diesem Notebook wird ein lokal betriebenes Large Language Model (LLM), konkret LLaMA 3.3 über die Ollama-Inferenzumgebung, eingesetzt, um die Aufgabe der Erkennung generischer Maskulina durchzuführen. Ziel ist es, die Ergebnisse des regelbasierten Systems mit einem kontextsensitiven, generativen Modell zu vergleichen und dadurch dessen Leistungsfähigkeit sowie typische Fehlerquellen besser einordnen zu können.

In [18]:
import pandas as pd
import requests
from tqdm import tqdm
import re
import json
from collections import defaultdict

# Enable tqdm in pandas
tqdm.pandas()

In [19]:
df = pd.read_pickle('local-store/final_data_frame_evaluation_one.pkl')

In [5]:
smaller_df = df.head(100)

In [9]:
type(smaller_df.iloc[0]["spacified_article"])

spacy.tokens.doc.Doc

In [11]:
# testing and experiments

prompt = "Was ist ein generisch-maskuliner Ausdruck?"
response = requests.post(
    "http://localhost:11434/api/generate",
    json={"model": "llama3", "prompt": prompt, "stream": False}
)
print(response.json()['response'])




A very specific question!

In German linguistics, a "generisch-maskulinier Ausdruck" (generic masculine expression) is a linguistic phenomenon where a noun or pronoun that refers to a person of unknown gender (i.e., a generic individual) is expressed in the masculine form.

In German, nouns and pronouns have different forms for each grammatical gender (masculine, feminine, neuter). While it might seem logical to use a neutral or gender-neutral term when referring to someone of unknown gender, this is not always the case. Instead, the masculine form is often used as a default or generic expression.

For example:

* "Ein Student" (a student) - here, "Student" is in the masculine form, even though it could refer to either a male or female student.
* "Jemand" (someone) - this pronoun is also in the masculine form and can be used to refer to anyone, regardless of gender.

This usage can be problematic because it implies that the default human being is male. In recent years, there has been a

In [24]:
print(list(smaller_df.columns))


['ArtikelId', 'Text', 'Text_unmarked', 'Sentences_marked', 'Sentences_unmarked', 'dependency_distance', 'marked_word_information', 'spacified_article', 'pipeline_result', 'result_counts', 'llm_result_basic_prompt_0', 'llm_result_basic_prompt_1', 'llm_result_basic_prompt_2', 'llm_result_basic_prompt_3', 'llm_result_basic_prompt_4', 'llm_result_basic_prompt_5', 'llm_result_basic_prompt_6', 'llm_result_basic_prompt_7', 'llm_result_basic_prompt_8', 'llm_result_basic_prompt_9', 'llm_result_prompt_with_role_0', 'llm_result_prompt_with_role_1', 'llm_result_prompt_with_role_2', 'llm_result_prompt_with_role_3', 'llm_result_prompt_with_role_4', 'llm_result_prompt_with_role_5', 'llm_result_prompt_with_role_6', 'llm_result_prompt_with_role_7', 'llm_result_prompt_with_role_8', 'llm_result_prompt_with_role_9', 'llm_result_prompt_with_example_0', 'llm_result_prompt_with_example_1', 'llm_result_prompt_with_example_2', 'llm_result_prompt_with_example_3', 'llm_result_prompt_with_example_4', 'llm_result_

In [12]:
#Diese Funktion führt den Grundsätzlichen request durch 
def query_ollama(prompt, model="llama3"):
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={"model": model, "prompt": prompt, "stream": False}
    )
    return response.json()["response"]

Die Funktion extract_gender_maskulin_from_text identifiziert potenzielle generisch-maskuline Begriffe in einem gegebenen Text mithilfe eines lokalen LLMs (z. B. LLaMA 3.3). Hierzu wird der Text satzweise in überlappende Fenster zerteilt, diese Abschnitte werden an das Sprachmodell geschickt und dessen Antwort (eine Liste erkannter Wörter) wird anschließend um die genauen Positionen der Begriffe im Originaltext ergänzt. Das Ergebnis ist eine Liste von Einträgen mit Wort und Positionsangabe.

In [13]:

def extract_gender_maskulin_from_text(text, spacydoc, prompt_start, model="llama3", window_size=3, step=1,):
    
    sätze = list(spacydoc.sents)

    def find_position_in_window(word, window_text, full_text):
        relative_start = window_text.find(word)
        if relative_start == -1:
            return None
        abs_start = full_text.find(window_text)
        if abs_start == -1:
            return None
        start = abs_start + relative_start
        return {"Wort": word, "from": start, "to": start + len(word)}

    def enrich_llm_results_with_positions(llm_results, window_text, full_text):
        enriched = []
        for entry in llm_results:
            word = entry["Wort"]
            pos_info = find_position_in_window(word, window_text, full_text)
            if pos_info:
                entry["from"] = pos_info["from"]
                entry["to"] = pos_info["to"]
            else:
                entry["from"] = None
                entry["to"] = None
            enriched.append(entry)
        return enriched

    all_results = []

    for i in range(0, len(sätze) - window_size + 1, step):
        fenster = sätze[i : i + window_size]
        window_text = "".join([s.text for s in fenster])

        prompt = f"""
{prompt_start}
Text:
{window_text}
"""

        try:
            response = query_ollama(prompt)
            match = re.search(r"\[.*?\]", response, re.DOTALL)
            if match:
                result = json.loads(match.group(0))
                result = enrich_llm_results_with_positions(result, window_text, text)
                all_results.extend(result)
        except Exception as e:
            print(f"Fehler bei Fenster {i}-{i+window_size}: {e}")

    return all_results



In [14]:
def remove_duplicates_dict_list(data):
    seen = set()
    unique = []
    for item in data:
        key = json.dumps(item, sort_keys=True)  # serialize dict als string
        if key not in seen:
            seen.add(key)
            unique.append(item)
    return unique


Verschiedene Prompts werden genutzt um die Resultate vergleichen zu können

In [15]:
name = 'name'
prompts = [
    { 'name': 'basic_prompt',
'prompt' : """Identifiziere alle generisch-maskulinen Ausdrücke im folgenden Text, die Textsprache ist Deutsch.
Gib eine JSON-Liste von Dictionaries zurück mit folgenden Felder:
- Wort"""},
           {'name': 'prompt_with_role',
            'prompt': """Du bist ein sprachwissenschaftliches Analysemodell. 
Finde alle generisch-maskulinen Personenbezeichnungen im folgenden Text, die Textsprache ist deutsch. 
Gib eine JSON-Liste von Dictionaries zurück mit folgenden Felder:
- Wort"""},
           {'name':'prompt_with_example',
            'prompt': """Identifiziere alle generisch-maskulinen Ausdrücke im folgenden Text, die Textsprache ist Deutsch.
Beispiel: In "Die Ärzte behandeln ihre Patienten." ist "Ärzte" und "Patienten" generisch.
Gib eine JSON-Liste von Dictionaries zurück mit folgenden Felder:
- Wort"""},
{'name': 'prompt_with_role_and_example', 
 'prompt': """Du bist ein sprachwissenschaftliches Analysemodell.
Identifiziere alle generisch-maskulinen Ausdrücke im folgenden Text, die Textsprache ist Deutsch.
Beispiel: In "Die Ärzte behandeln ihre Patienten." ist "Ärzte" und "Patienten" generisch.
Gib eine JSON-Liste von Dictionaries zurück mit folgenden Felder:
- Wort"""},
{
'name':'prompt_with_hints',
    'prompt': """Identifiziere alle generisch-maskulinen Ausdrücke im folgenden Text, die Textsprache ist Deutsch. Hinweise: 
Achte darauf ob das Wort einen Bezug zu einer eindeutig männlichen Person hat. Achte darauf das es überhaupt möglich ist das Wort zu gendern.
Ignoriere maskuline begriffe die nicht generisch sind wie bspw. Mut.
Gib eine JSON-Liste von Dictionaries zurück mit folgenden Felder:
- Wort"""}]




In [25]:
for prompt in prompts:
     for i in range(10):
         print(prompt[name])
         smaller_df[f'llm_result_{prompt[name]}_{i}']= smaller_df.progress_apply(
         lambda row: extract_gender_maskulin_from_text( row['Text_unmarked'], row["spacified_article"], prompt['prompt']),
         axis=1
         )

prompt_with_hints


  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

Fehler bei Fenster 14-17: Expecting ',' delimiter: line 6 column 18 (char 53)
Fehler bei Fenster 17-20: Expecting ',' delimiter: line 6 column 21 (char 61)
Fehler bei Fenster 27-30: Expecting ',' delimiter: line 6 column 21 (char 72)


  2%|█▌                                                                              | 2/100 [03:09<2:34:47, 94.77s/it]

Fehler bei Fenster 9-12: Expecting ',' delimiter: line 6 column 26 (char 61)
Fehler bei Fenster 10-13: Expecting property name enclosed in double quotes: line 3 column 43 (char 66)
Fehler bei Fenster 37-40: Expecting ',' delimiter: line 6 column 24 (char 56)


...

 98%|█████████████████████████████████████████████████████████████████████████████▍ | 98/100 [2:06:37<02:51, 85.68s/it]

Fehler bei Fenster 8-11: Expecting ',' delimiter: line 6 column 24 (char 62)


100%|██████████████████████████████████████████████████████████████████████████████| 100/100 [2:09:49<00:00, 91.58s/it]

Fehler bei Fenster 11-14: Expecting ',' delimiter: line 6 column 19 (char 57)


100%|██████████████████████████████████████████████████████████████████████████████| 100/100 [2:10:40<00:00, 78.40s/it]


In [26]:
#import helperfunctions:         
from helperfunctions import  find_in_result, update_counts


In [27]:
# Da an dieser Stelle kein Interesse an den Detailresultaten pro Zeile besteht reicht eine vereinfachte Version der Funktion aus Evaluation.py

def compare_marked_to_pipeline(marked, llm_result, spacify, counter):
    seen_positions = set()

    # 1. Verarbeite alle markierten Wörter
    for entry in marked: 
        word_in_pipeline_result = find_in_result(llm_result, entry['from'], entry['to'])
        begin_ = [x for x in spacify if x.idx == entry["from"]]
        if not begin_:
            print(f"could not find: {entry}")
            continue

        _word = begin_[0]
        seen_positions.add((entry['from'], entry['to']))
        needs_to_be_gendered = entry['ntbg']  # Erwartung laut Markierung

        # Systemeinschätzung
        if word_in_pipeline_result:
            was_correct = True == needs_to_be_gendered
        else:
            was_correct = not needs_to_be_gendered

        is_noun = _word.pos_ == 'NOUN'
        number = _word.morph.get("Number")
        is_singular = number[0] == "Sing" if number else None

        update_counts(counter, needs_to_be_gendered, was_correct, is_noun, is_singular, True)

    # 2. Verarbeite False Positives aus Pipeline (nicht in Markierung enthalten)
    for entry in llm_result:
        pos = (entry['from'], entry['to'])
        if pos in seen_positions:
            continue  # bereits abgeglichen

        begin_ = [x for x in spacify if x.idx == entry["from"]]
        if not begin_:
            print(f"could not find in spacify: {entry}")
            continue

        _word = begin_[0]
        is_noun = _word.pos_ == 'NOUN'
        number = _word.morph.get("Number")
        is_singular = number[0] == "Sing" if number else None

        # False Positive – systematisch falsch markiert
        update_counts(counter, False, False, is_noun, is_singular, False)

    return counter


In [28]:
markings = df['marked_word_information'].iloc[5]
cleanResponse = remove_duplicates_dict_list(response)
countsnew = compare_marked_to_pipeline(markings, cleanResponse, spacy_information, defaultdict(int) )

TypeError: Object of type bytes is not JSON serializable

In [29]:
#smaller_df.to_pickle('local-store/llm_benchmarking_prompts.pkl') #wieder lokales zwischenspeichern der Ergebnisse
smaller_df = pd.read_pickle('local-store/llm_benchmarking_prompts.pkl')

In [36]:
all_results = defaultdict(dict)

for prompt in prompts:
    prompt_name = prompt[name]
    for i in range(10):
        counter = defaultdict(int)

        # Analyse mit deiner Funktion
        _ = smaller_df.progress_apply(
            lambda row: compare_marked_to_pipeline(
                row['marked_word_information'],
                remove_duplicates_dict_list(row[f'llm_result_{prompt_name}_{i}']),
                row['spacified_article'],
                counter
            ),
            axis=1
        )

        # Speichere unter: prompt_name → i → counter
        all_results[prompt_name][str(i)] = dict(counter)  # defaultdict zu normalem dict konvertieren

# Schreibe alles auf einmal als JSON-Datei
with open('llmResult.json', 'w', encoding='utf-8') as f:
    json.dump(all_results, f, ensure_ascii=False, indent=2)

 25%|████████████████████                                                            | 25/100 [00:00<00:00, 120.37it/s]

could not find in spacify: {'Wort': 'er', 'from': 1378, 'to': 1380}
could not find in spacify: {'Wort': 'ein', 'from': 1366, 'to': 1369}
could not find in spacify: {'Wort': 'ein ehemaliger Geschäftsführer', 'from': None, 'to': None}
could not find in spacify: {'Wort': 'er', 'from': 2982, 'to': 2984}
could not find in spacify: {'Wort': 'ihn', 'from': None, 'to': None}
could not find in spacify: {'Wort': 'er', 'from': 3066, 'to': 3068}
could not find in spacify: {'Wort': 'er', 'from': 3692, 'to': 3694}
could not find in spacify: {'Wort': 'er', 'from': 3729, 'to': 3731}
could not find in spacify: {'Wort': 'ein Macher', 'from': None, 'to': None}
could not find in spacify: {'Wort': 'er', 'from': 4021, 'to': 4023}
could not find in spacify: {'Wort': 'er', 'from': 4197, 'to': 4199}
could not find in spacify: {'Wort': 'er', 'from': 4824, 'to': 4826}
could not find in spacify: {'Wort': 'ein', 'from': 4992, 'to': 4995}
could not find in spacify: {'Wort': 'er', 'from': 5047, 'to': 5049}
could not

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 270.27it/s]


could not find in spacify: {'Wort': 'Schnellzug', 'from': None, 'to': None}
could not find in spacify: {'Wort': 'der', 'from': 1758, 'to': 1761}
could not find in spacify: {'Wort': 'Staatschefs', 'from': 383, 'to': 394}
could not find in spacify: {'Wort': 'Parlamentarischer Staatssekretär', 'from': None, 'to': None}
could not find in spacify: {'Wort': 'Parlamentarischer', 'from': None, 'to': None}
could not find in spacify: {'Wort': 'Regierungskreise', 'from': 259, 'to': 275}
could not find in spacify: {'Wort': 'Polizeichef', 'from': 6, 'to': 17}
could not find in spacify: {'Wort': 'ihn', 'from': None, 'to': None}
could not find in spacify: {'Wort': 'designierte Ministerpräsident', 'from': None, 'to': None}
could not find in spacify: {'Wort': 'Israel', 'Funktion': 'generisch-maskulin', 'from': None, 'to': None}
could not find in spacify: {'Wort': 'Der Angriff', 'Funktion': 'generisch-maskulin', 'from': None, 'to': None}
could not find in spacify: {'Wort': 'Gericht', 'from': None, 'to':

 10%|████████                                                                         | 10/100 [00:00<00:01, 86.73it/s]

...

could not find in spacify: {'Wort': 'Rebellierten', 'from': None, 'to': None}
could not find in spacify: {'Wort': 'Männer', 'from': None, 'to': None}
could not find in spacify: {'Wort': 'Revutionsführer', 'from': None, 'to': None}
could not find in spacify: {'Wort': 'Revolutionsgerichten', 'from': None, 'to': None}
could not find in spacify: {'Wort': 'Revolutionswächtern', 'from': None, 'to': None}
could not find in spacify: {'Wort': 'Richter', 'from': None, 'to': None}
could not find in spacify: {'Wort': 'Revolutionäre Führer', 'from': None, 'to': None}
could not find in spacify: {'Wort': 'Asoziale', 'from': None, 'to': None}
could not find in spacify: {'Wort': 'Dealer', 'from': None, 'to': None}
could not find in spacify: {'Wort': 'Messerstecher', 'from': None, 'to': None}
could not find in spacify: {'Wort': 'Schmarotzer', 'from': None, 'to': None}
could not find in spacify: {'Wort': 'Geier', 'from': None, 'to': None}
could not find in spacify: {'Wort': 'sie', 'from': None, 'to': Non

 28%|██████████████████████▍                                                         | 28/100 [00:00<00:00, 134.31it/s]


could not find in spacify: {'Wort': 'Politiker', 'from': 3900, 'to': 3909}
could not find in spacify: {'Wort': 'der', 'from': 4188, 'to': 4191}
could not find in spacify: {'Wort': 'Eigentümer', 'from': None, 'to': None}
could not find in spacify: {'Wort': 'Bauern', 'from': None, 'to': None}
could not find in spacify: {'Wort': 'er', 'from': 6232, 'to': 6234}
could not find in spacify: {'Wort': 'er', 'from': 5, 'to': 7}
could not find in spacify: {'Wort': 'des', 'from': None, 'to': None}
could not find in spacify: {'Wort': 'sie', 'from': None, 'to': None}
could not find in spacify: {'Wort': 'Er', 'from': None, 'to': None}
could not find in spacify: {'Wort': 'der', 'from': None, 'to': None}
could not find in spacify: {'Wort': 'der beste', 'from': None, 'to': None}
could not find in spacify: {'Wort': 'Britten', 'from': None, 'to': None}
could not find in spacify: {'Wort': 'Höchststimmrecht', 'from': 759, 'to': 775}
could not find in spacify: {'Wort': 'Politiker', 'from': None, 'to': None}

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 287.48it/s]


could not find in spacify: {'Wort': 'er', 'from': 1437, 'to': 1439}
could not find in spacify: {'Wort': 'er', 'from': 1508, 'to': 1510}
could not find in spacify: {'Wort': 'er', 'from': 1543, 'to': 1545}
could not find in spacify: {'Wort': 'Landesvorsitzende', 'from': 236, 'to': 253}
could not find in spacify: {'Wort': 'er', 'from': 1112, 'to': 1114}
could not find in spacify: {'Wort': 'Kandidaten', 'from': None, 'to': None}
could not find in spacify: {'Wort': 'Kandidat', 'from': None, 'to': None}
could not find in spacify: {'Wort': 'er', 'from': 2462, 'to': 2464}
could not find in spacify: {'Wort': 'er', 'from': 3335, 'to': 3337}
could not find in spacify: {'Wort': 'Grüner', 'from': None, 'to': None}
could not find in spacify: {'Wort': 'er', 'from': 3775, 'to': 3777}
could not find in spacify: {'Wort': 'Ihr', 'from': None, 'to': None}
could not find in spacify: {'Wort': 'einigen', 'from': None, 'to': None}
could not find in spacify: {'Wort': 'Ministerium', 'from': None, 'to': None}
c

An dieser Stelle wird auf komplizierte Ausgabenlogik verzichtet und die Zahlen aus einem einfach Print übernommen.

In [37]:
with open('llmResult.json', 'r', encoding='utf-8') as f:
    all_results = json.load(f)

averages_per_prompt = {}
best_runs = {}
worst_runs = {}

primary_metric = "correct"  # Alternativ: "incorrect"

for prompt_name, iterations in all_results.items():
    metric_sums = defaultdict(int)
    num_iterations = 0

    best_value = -float('inf')
    worst_value = float('inf')
    best_run = None
    worst_run = None

    for run_id, metrics in iterations.items():
        for key, value in metrics.items():
            metric_sums[key] += value

        current_value = metrics.get(primary_metric, 0)

        if current_value > best_value:
            best_value = current_value
            best_run = (run_id, metrics)

        if current_value < worst_value:
            worst_value = current_value
            worst_run = (run_id, metrics)

        num_iterations += 1

    averages = {key: metric_sums[key] / num_iterations for key in metric_sums}
    averages_per_prompt[prompt_name] = averages
    best_runs[prompt_name] = best_run
    worst_runs[prompt_name] = worst_run


In [38]:
output = {
    "averages": averages_per_prompt,
    "best_runs": best_runs,
    "worst_runs": worst_runs
}

with open("llmResult_summary.json", "w", encoding="utf-8") as f:
    json.dump(output, f, ensure_ascii=False, indent=2)